## Working With Dummy Data 

In [34]:
import folium # type: ignore
import geopandas as gpd # type: ignore
from shapely.geometry import Point # type: ignore
import random 

random.seed(100)


In [35]:
map = "../data/maps/ontario.geojson"

gdf = gpd.read_file(map).to_crs(epsg=3857)
dummy_map = gdf.explode(ignore_index=True).iloc[[2]]
dummy_map['Boundary'] = 'Prince-Edward-County'

In [36]:
def monte_carlo_sampling(polygon, num_points=10):
    """
    Generates a specified number of random points within a given polygon.

    This function samples random points by generating coordinates within the 
    bounding box of the polygon and checking if they fall inside the polygon. 
    It continues generating points until the required number is reached.

    Parameters:
    - polygon (shapely.geometry.Polygon): The polygon within which points should be sampled.
    - num_points (int, optional): The number of random points to generate. Default is 100.

    Returns:
    - list of shapely.geometry.Point: A list of points inside the polygon.

    Note:
    - This method uses a rejection sampling approach, which may be inefficient 
      for complex or irregularly shaped polygons with large holes.
    """
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < num_points:
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            points.append(random_point)
    return points

In [37]:
def plot_sampled_points(sampled_points, base_map, map_object):
    """
    Plots sampled points on a Folium map as a separate layer.

    This function takes a list of sampled points, converts them into a GeoDataFrame, 
    ensures they have the correct CRS (EPSG:4326) for mapping, and adds them to 
    the provided Folium map as a `FeatureGroup`.

    Parameters:
    - sampled_points (list of shapely.geometry.Point): List of sampled points to be plotted.
    - base_map (geopandas.GeoDataFrame): The base map containing spatial reference (CRS).
    - map_object (folium.Map): The Folium map to which the points will be added.

    Returns:
    - folium.Map: The updated map with the sampled points layer.

    Note:
    - The function automatically reprojects the points and the base map to EPSG:4326 if needed.
    - The added layer can be toggled using the LayerControl feature.
    """

    # Convert points to GeoDataFrame
    points_gdf = gpd.GeoDataFrame(geometry=sampled_points, crs=base_map.crs)

    # Ensure CRS is EPSG:4326 for mapping
    if points_gdf.crs.to_string() != "EPSG:4326":
        points_gdf = points_gdf.to_crs(epsg=4326)
    if base_map.crs.to_string() != "EPSG:4326":
        base_map = base_map.to_crs(epsg=4326)

    # Create FeatureGroup for points
    point_layer = folium.FeatureGroup(name="Sampled Points", overlay=True, control=True)

    # Add each point as a CircleMarker
    for _, row in points_gdf.iterrows():
        point = row.geometry
        folium.CircleMarker(
            location=[point.y, point.x],  # (lat, lon)
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(point_layer)

    # Add the points layer to the map
    point_layer.add_to(map_object)

    # Ensure LayerControl is added for toggling layers
    folium.LayerControl(collapsed=False).add_to(map_object)

    return map_object  # Return the updated map

In [38]:
m = folium.Map(location=[44, -77.2], zoom_start=11)

boundary_layer = folium.FeatureGroup(name="Prince-Edward-County Boundary", 
                                     overlay=True, control=True)

folium.GeoJson(
    dummy_map, 
    name="Prince-Edward-County Boundary", 
    style_function=lambda feature: {"color": "black", "weight": 2, "fillOpacity": 0},
    tooltip=folium.GeoJsonTooltip(fields=["Boundary"])
).add_to(boundary_layer)

boundary_layer.add_to(m)

sampled_points = monte_carlo_sampling(dummy_map.geometry.iloc[0], num_points=10)
m = plot_sampled_points(sampled_points, dummy_map, m)
m

In [28]:
for key in m._children:
    print(key)

openstreetmap
feature_group_7d728583ff2eccac11555ec26bb571ca
feature_group_361ce2a2709bb1e075221d6e45ec76c1
layer_control_7d24edd0f10805e1c2573eee0bff538b
